<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/кейсы%20по%20анализу%20данных/Загрузка_данных.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

In [164]:
import numpy as np
import pandas as pd

import datetime
from datetime import datetime, date

## Поворот

Считываем данные акселерометра:

In [198]:
sensor_readings = pd.read_csv("Data/accel.csv")
sensor_readings

,interval,axis,reading
0,0,X,0.0
1,0,Y,0.5
2,0,Z,1.0
3,1,X,0.1
4,1,Y,0.4
5,1,Z,0.9
6,2,X,0.2
7,2,Y,0.3
8,2,Z,0.8
9,3,X,0.3


Извлекаем показания по оси X:

In [199]:
sensor_readings[sensor_readings['axis'] == 'X']

,interval,axis,reading
0,0,X,0.0
3,1,X,0.1
6,2,X,0.2
9,3,X,0.3


Поворачиваем данные. Интервалы становятся индексом, столбцы - это оси, а показания - значения столбцов:

In [200]:
sensor_readings.pivot(index='interval', 
                     columns='axis', 
                     values='reading')

axis,X,Y,Z
interval,,,
0,0.0,0.5,1.0
1,0.1,0.4,0.9
2,0.2,0.3,0.8
3,0.3,0.2,0.7


## Состыковка с помощью неиерархических индексов

Создаем простой датафрейм с одним столбцом:

In [201]:
df = pd.DataFrame({'a': [1, 2]}, index={'one', 'two'})
df

,a
two,1
one,2


Помещаем столбец в еще один уровень индекса строк результатом становится объект Series, в которой значения можно просмотреть с помощью мультииндекса:

In [202]:
stacked1 = df.stack()
stacked1

two  a    1
one  a    2
dtype: int64

Ищем значение для 'one'/'a', передав кортеж в индексатор:

In [203]:
stacked1[('one', 'a')]

2

Создаем датафрейм с двумя столбцами:

In [204]:
df = pd.DataFrame({'a': [1, 2],
                   'b': [3, 4]}, 
                  index={'one', 'two'})
df

,a,b
two,1,3
one,2,4


Помещаем оба столбца в отдельный уровень индекса:

In [205]:
stacked2 = df.stack()
stacked2

two  a    1
     b    3
one  a    2
     b    4
dtype: int64

Ищем значение с помощью индекса 'one' / 'b':

In [206]:
stacked2[('one', 'b')]

4

## Расстыковка с помощью иерархических индексов

Создаем две копии данных акселерометра, по одной для каждого пользователя:

In [207]:
user1 = sensor_readings.copy()
user2 = sensor_readings.copy()

Добавляем столбец who в каждую копию:

In [207]:
user1['who'] = 'Mike'
user2['who'] = 'Mikael'

Давайте отмасштабируем данные user2:

In [207]:
user2['reading'] *= 100

И организуем данные так, чтобы получить иерархический индекс строк:

In [207]:
multi_user_sensor_data = pd.concat([user1, user2]) \
            .set_index(['who', 'interval', 'axis'])
multi_user_sensor_data

reading
who    interval axis         
Mike   0        X         0.0
                Y         0.5
                Z         1.0
       1        X         0.1
                Y         0.4
                Z         0.9
       2        X         0.2
                Y         0.3
                Z         0.8
       3        X         0.3
                Y         0.2
                Z         0.7
Mikael 0        X         0.0
                Y        50.0
                Z       100.0
       1        X        10.0
                Y        40.0
                Z        90.0
       2        X        20.0
                Y        30.0
                Z        80.0
       3        X        30.0
                Y        20.0
                Z        70.0

Извлекаем показания, относящиеся к пользователю Mike, с помощью индекса:

In [208]:
multi_user_sensor_data.loc['Mike']

reading
interval axis         
0        X         0.0
         Y         0.5
         Z         1.0
1        X         0.1
         Y         0.4
         Z         0.9
2        X         0.2
         Y         0.3
         Z         0.8
3        X         0.3
         Y         0.2
         Z         0.7

Извлекаем все показания по всем осям и по всем пользователям в интервале 1:

In [209]:
multi_user_sensor_data.xs(1, level='interval')

reading
who    axis         
Mike   X         0.1
       Y         0.4
       Z         0.9
Mikael X        10.0
       Y        40.0
       Z        90.0

Выполняем расстыковку, в результате самый внутренний уровень индекса строк (уровень axis) стал уровнем индекса столбцов:

In [210]:
multi_user_sensor_data.unstack()

reading             
axis                  X     Y      Z
who    interval                     
Mikael 0            0.0  50.0  100.0
       1           10.0  40.0   90.0
       2           20.0  30.0   80.0
       3           30.0  20.0   70.0
Mike   0            0.0   0.5    1.0
       1            0.1   0.4    0.9
       2            0.2   0.3    0.8
       3            0.3   0.2    0.7

Выполняем расстыковку по уровню 0:

In [211]:
multi_user_sensor_data.unstack(level=0)

reading     
who            Mikael Mike
interval axis             
0        X        0.0  0.0
         Y       50.0  0.5
         Z      100.0  1.0
1        X       10.0  0.1
         Y       40.0  0.4
         Z       90.0  0.9
2        X       20.0  0.2
         Y       30.0  0.3
         Z       80.0  0.8
3        X       30.0  0.3
         Y       20.0  0.2
         Z       70.0  0.7

Выполняем расстыковку уровней who и axis:

In [212]:
unstacked = multi_user_sensor_data.unstack(['who', 'axis'])
unstacked

reading                              
who         Mike           Mikael             
axis           X    Y    Z      X     Y      Z
interval                                      
0            0.0  0.5  1.0    0.0  50.0  100.0
1            0.1  0.4  0.9   10.0  40.0   90.0
2            0.2  0.3  0.8   20.0  30.0   80.0
3            0.3  0.2  0.7   30.0  20.0   70.0

И, конечно, мы можем выполнить состыковку уровней, которые расстыковали выполняем состыковку уровня who:

In [213]:
unstacked.stack(level='who')

reading             
axis                  X     Y      Z
interval who                        
0        Mikael     0.0  50.0  100.0
         Mike       0.0   0.5    1.0
1        Mikael    10.0  40.0   90.0
         Mike       0.1   0.4    0.9
2        Mikael    20.0  30.0   80.0
         Mike       0.2   0.3    0.8
3        Mikael    30.0  20.0   70.0
         Mike       0.3   0.2    0.7

## Расплавление

Продемонстрируем расплавление с помощью этого датафрейма:

In [214]:
data = pd.DataFrame({'Name' : ['Mike', 'Mikael'],
                     'Height' : [6.1, 6.0],
                     'Weight' : [220, 185]})
data

,Name,Height,Weight
0,Mike,6.1,220
1,Mikael,6.0,185


Расплавляем датафрейм, используем Name в качестве идентификатора, а столбцы  Height and Weight в качестве переменных:

In [215]:
pd.melt(data, 
        id_vars=['Name'],
        value_vars=['Height', 'Weight'])

,Name,variable,value
0,Mike,Height,6.1
1,Mikael,Height,6.0
2,Mike,Weight,220.0
3,Mikael,Weight,185.0
